In [5]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
print(selenium.__version__)

4.21.0


In [2]:
# Read data from excel
# Read the Excel file
file_path = '../migration_db2.xlsx'  # Replace with your file path
sheet_name = 'Sheet3'  # Replace with your sheet name
column_name_city = 'city_scrape'  # Replace with your column name
column_name_country = 'country_scrape'  # Replace with your column name

# Load the Excel file
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert the column to a list
city_list = df[column_name_city].tolist()
country_list = df[column_name_country].tolist()

# final list
city_names = []
city_names_bis = []
for idx,city in enumerate(city_list):
    city_names.append(city_list[idx])
    city_names_bis.append(str(city_list[idx])+'-'+str(country_list[idx]))

In [3]:
# Initialize WebDriver

# Set up Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up the WebDriver with the Chrome options
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()  # Assuming you're using Chrome

# Base URL
url_base = 'https://www.numbeo.com/quality-of-life/in/'

# Initialize an empty DataFrame
df_capital_all = pd.DataFrame(columns=[
    'index',
    'city',
    'climate_index',
    'cost_of_living_index',
    'health_care_index',
    'pollution_index',
    'property_price_to_income_ratio',
    'purchasing_power_index',
    'safety_index',
    'traffic_commute_time_index',
    'quality_of_life_index'
])

In [4]:
# Fetch data using Selenium

for idx,city in enumerate(city_names):
    
    try:

        url_city = url_base + city_names[idx]
        driver.get(url_city)

        # Find Table element
        tbl_quality = driver.find_element(By.XPATH, '/html/body/div[2]/table')  # if there's no ID, always a good idea to copy xpath

        
        # Parse Table element into pandas DataFrame
        tbl_quality_html = tbl_quality.get_attribute('outerHTML')
        soup = BeautifulSoup(tbl_quality_html, 'html.parser')

        # Find all table rows in the parsed HTML
        tbl_quality_rows = soup.find_all('tr')

        # Initialize data list
        data_cost = []
        for row in tbl_quality_rows:
                row_data = []
                cells = row.find_all(['td', 'th'])
                for cell in cells:
                    row_data.append(cell.get_text().strip())
                data_cost.append(row_data)

        # Extract header and data rows separately
        header = ['kpi_name','kpi_value','kpi_hml']

        # Create DataFrame
        df_capital = pd.DataFrame(data_cost, columns=header)

        # Clean data
        df_capital['Index'] = idx
        df_capital['City'] = city_names[idx]
        df_capital.drop(columns=['kpi_hml'], inplace=True)
        df_capital['kpi_value'] = pd.to_numeric(df_capital['kpi_value'], errors='coerce')
        # df_capital = df_capital[~df_capital['kpi_name'].isna()].copy()
        df_capital.drop(8, inplace=True)


        # Transpose df
        df_capital_t = df_capital.pivot_table(index=['Index','City'], columns='kpi_name', values='kpi_value', dropna=False)
        df_capital_t.reset_index(inplace=True)
        df_capital_t.columns = df_capital_all.columns

        # Append data to complete DF
        df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)

    except:

        try:

            url_city = url_base + city_names_bis[idx]
            driver.get(url_city)

            # Find Table element
            tbl_quality = driver.find_element(By.XPATH, '/html/body/div[2]/table')  # if there's no ID, always a good idea to copy xpath

            
            # Parse Table element into pandas DataFrame
            tbl_quality_html = tbl_quality.get_attribute('outerHTML')
            soup = BeautifulSoup(tbl_quality_html, 'html.parser')

            # Find all table rows in the parsed HTML
            tbl_quality_rows = soup.find_all('tr')

            # Initialize data list
            data_cost = []
            for row in tbl_quality_rows:
                    row_data = []
                    cells = row.find_all(['td', 'th'])
                    for cell in cells:
                        row_data.append(cell.get_text().strip())
                    data_cost.append(row_data)

            # Extract header and data rows separately
            header = ['kpi_name','kpi_value','kpi_hml']

            # Create DataFrame
            df_capital = pd.DataFrame(data_cost, columns=header)

            # Clean data
            df_capital['Index'] = idx
            df_capital['City'] = city_names_bis[idx]
            df_capital.drop(columns=['kpi_hml'], inplace=True)
            df_capital['kpi_value'] = pd.to_numeric(df_capital['kpi_value'], errors='coerce')
            # df_capital = df_capital[~df_capital['kpi_name'].isna()].copy()
            df_capital.drop(8, inplace=True)


            # Transpose df
            df_capital_t = df_capital.pivot_table(index=['Index','City'], columns='kpi_name', values='kpi_value', dropna=False)
            df_capital_t.reset_index(inplace=True)
            df_capital_t.columns = df_capital_all.columns

            # Append data to complete DF
            df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)

        except:
             
             # Append NANs
            emtpy_dict = {
            'index':[idx]
            ,'city':[city_names[idx]]
            ,'climate_index':[pd.NA]
            , 'cost_of_living_index':[pd.NA]
            ,'health_care_index':[pd.NA]
            ,'pollution_index':[pd.NA]
            ,'property_price_to_income_ratio':[pd.NA]
            ,'purchasing_power_index':[pd.NA]
            ,'safety_index':[pd.NA]
            ,'traffic_commute_time_index':[pd.NA]
            ,'quality_of_life_index':[pd.NA]
            }
            df_capital_t = pd.DataFrame.from_dict(emtpy_dict)
            df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)


C:\Users\paude\AppData\Local\Temp\ipykernel_10792\3857597324.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)
C:\Users\paude\AppData\Local\Temp\ipykernel_10792\3857597324.py:120: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)
C:\Users\paude\AppData\Local\Temp\ipykernel_10792\3857597324.py:120: FutureWarning: The behavior of DataFrame concatenation with empty or a

In [41]:
# Merge Net Migration from original df

df_capital_all.reset_index(drop=True, inplace=True)
df_capital_all.drop_duplicates(subset='city', inplace=True)
df_capital_all = df_capital_all.merge(df['city_net_migration'], left_index=True, right_index=True, how='left')

# Drop entries with NAN in all indexes
index_cols = [
    'climate_index'	
    ,'cost_of_living_index'
    ,'health_care_index'
    ,'pollution_index'
    ,'property_price_to_income_ratio'
    ,'purchasing_power_index'
    ,'safety_index'
    ,'traffic_commute_time_index'
    ,'quality_of_life_index'
    ]

df_capital_all = df_capital_all.dropna(subset=index_cols, how='all')
df_capital_all.head(20)

,index,city,climate_index,cost_of_living_index,health_care_index,pollution_index,property_price_to_income_ratio,purchasing_power_index,safety_index,traffic_commute_time_index,quality_of_life_index,city_net_migration_x,city_net_migration_y
0,3,Kabul,NaN,NaN,26.16,89.71,9.61,NaN,22.09,56.17,NaN,-427.3156,-9.6093
1,6,Annaba-Algeria,97.28,NaN,76.85,95.69,12.93,NaN,36.56,15.00,NaN,-102.9436,-50.0000
2,19,Bahia-Blanca-Argentina,91.54,NaN,67.59,63.45,20.00,NaN,54.15,30.00,NaN,-30.1572,-26.3312
3,20,Buenos-Aires,98.28,32.65,68.39,51.76,23.34,32.49,36.56,49.65,105.45,-1671.0000,-427.3156
4,24,Ensenada,96.82,NaN,93.60,43.83,NaN,NaN,60.68,37.00,NaN,-5.4463,-61.4254
5,26,La-Paz,55.46,30.10,47.01,75.36,12.05,40.49,39.56,52.67,81.64,-2.4307,-46.9247
6,27,La-Plata-Argentina,97.52,NaN,60.19,60.20,7.00,NaN,31.53,39.11,NaN,-19.3144,-102.9436
7,28,Mendoza-Argentina,87.76,26.39,58.33,36.18,11.61,50.77,34.42,36.50,133.49,-11.5041,-116.2580
8,30,Rosario,96.13,27.80,75.53,42.97,12.70,52.51,24.44,25.61,138.54,-127.6000,-65.4344
9,31,San-Juan,71.25,73.68,62.98,48.88,6.96,67.44,31.93,36.25,126.84,-47.1389,-62.0536


In [42]:
len(df_capital_all['city'].unique())

390

In [43]:
df_capital_all.shape

(390, 13)

In [40]:
df[df['city_scrape']=='Buenos-Aires'].head()

,City,Country,Lat,Lng,iso2,iso3,Population,MIGRATION AS A % OF POPULATION/ country,total_net_migration_2022 / country,net_mig_per_thousand_people_2023 / country,CITY,Data Year 2,Data Year 3,Population Growth Rate (PGR) Year 2 to Year 3 (%),Net Migration Rate per person: \n(Net migration per 1000 people/1000),city_net_migration,City Population Adjusted\n(Pcity x (1+(PGR/100),Adjusted City Net Migration:,country_scrape,city_scrape
20,Buenos Aires,Argentina,-34.6033,-58.3817,AR,ARG,16710000.0,5.048542,3886,-0.1,Buenos Aires,2010.0,2020.0,0.382962,-0.0001,-1671.0000,1.677399e+07,-1677.399298,Argentina,Buenos-Aires
104,Buenos Aires,Brazil,-7.7258,-35.3269,BR,BRA,12808.0,0.507956,6425,-0.2,Buenos Aires,2010.0,2020.0,0.382962,-0.0002,-2.5616,1.285705e+04,-2.571410,Brazil,Buenos-Aires
211,Buenos Aires,Colombia,2.9167,-76.6667,CO,COL,35197.0,3.744664,-167924,-1.9,Buenos Aires,2010.0,2020.0,0.382962,-0.0019,-66.8743,3.533179e+04,-67.130403,Colombia,Buenos-Aires
250,Buenos Aires,Costa Rica,9.1985,-83.2816,CR,CRI,29454.0,10.222170,3850,2.0,Buenos Aires,2010.0,2020.0,0.382962,0.0020,58.9080,2.956680e+04,59.133595,Costa-Rica,Buenos-Aires


In [36]:
df_capital_all.shape

(516, 12)

In [34]:
df_capital_all.to_csv('test_out_final_3.csv')

In [44]:
# Define the file path
file_path = r'C:\Users\paude\Documents\00. Backup 2023\EXTERIOR\MASTER\MA IAAC\03. Cursada\01. Modulo 03\S01 DE\Class Project\DE_Team\Scraped_csvs\data_migration.csv'

# Export DataFrame to a CSV file
df_capital_all.to_csv(file_path, index=False)

print("DataFrame exported successfully to:", file_path)

DataFrame exported successfully to: C:\Users\paude\Documents\00. Backup 2023\EXTERIOR\MASTER\MA IAAC\03. Cursada\01. Modulo 03\S01 DE\Class Project\DE_Team\Scraped_csvs\data_migration.csv
